In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
!pip install tensorflow
!pip install pydub
!apt-get install ffmpeg

In [ ]:
from IPython.display import Image
import time
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2
from IPython.display import Audio, display


audio_file = "/content/drive/MyDrive/RBE Model/example.mp3" # replace with your file name
audio = AudioSegment.from_file(audio_file)


i = 1
model = tf.keras.models.load_model("/content/drive/MyDrive/RBE Model/model.h5")
try:
  while True:
    filename = take_photo("/content/sample_data/{}.png".format(i))
    print('Saved to {}'.format(filename))
    img=cv2.imread(str(filename))
    plt.imshow(img)
    resized_image=cv2.resize(img,(224,224))
    resized_image = np.array(resized_image)/255
    resized_image = np.expand_dims(resized_image, axis=0)
    list_pred = model.predict(resized_image)
    print(list_pred)
    list_pred = list_pred.tolist()
    if list_pred[0].index(max(list_pred[0])) == 0 :
      print("With Social Distance")
      display(Image(filename))
    else :
      print("Without Social Distance")
      display(Image(filename))
      display(Audio(audio_file, autoplay=True))
    i = i+1
    time.sleep(5)
  
  # Show the image which was just taken.
  
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))